In [1]:
import os

from platform import python_version
import warnings
import time
import datetime as dt
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import multiprocessing as mp
import shutil

from imblearn.keras import BalancedBatchGenerator
from imblearn.under_sampling import RandomUnderSampler

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.densenet import DenseNet121, preprocess_input, decode_predictions
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.utils import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.initializers import *
from tensorflow.keras import regularizers

import pandas as pd
import numpy as np
import seaborn as sns
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from PIL import Image
import xml.etree.ElementTree as ET
import psutil
import random

warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
img_size = 96
epochs = 75
batch_size = 32
testsplit = .2
targetx = 96
targety = 96
learning_rate = 0.0001
classes = 7
seed = 23
print(seed)


23


In [3]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from collections import Counter

model_name = 'INV3Balancedmodel.keras'

# Define two different ImageDataGenerators with their respective directories
datagen1 = ImageDataGenerator(
        rescale=1./255,
        brightness_range=[0.9,1.1],
        horizontal_flip=True,
        fill_mode='nearest',
        preprocessing_function=preprocess_input
)

datagen2 = ImageDataGenerator(
        rescale=1./255,
        brightness_range=[0.9,1.1],
        horizontal_flip=True,
        fill_mode='nearest',
        preprocessing_function=preprocess_input
)

train_dir1 = 'KDEFMF/MaleBalancedTraining'
train_dir2 = 'KDEFMF/FemaleBalancedTraining'
val_dir1 = 'KDEFMF/MaleBalancedValidation'
val_dir2 = 'KDEFMF/FemaleBalancedValidation'

# Create two separate generators using different directories
train_generator1 = datagen1.flow_from_directory(
    train_dir1,
    target_size=(targetx,targety),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    seed=seed
)
validation_generator1 = datagen1.flow_from_directory(
    val_dir1,
    target_size=(targetx,targety),
    batch_size=batch_size,
    class_mode='categorical',
)
validation_generator2 = datagen2.flow_from_directory(
    val_dir2,
    target_size=(targetx,targety),
    batch_size=batch_size,
    class_mode='categorical',
)

train_generator2 = datagen2.flow_from_directory(
    train_dir2,
    target_size=(targetx,targety),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    seed=seed
)

total_train_samples = len(train_generator1.filenames) + len(train_generator2.filenames)
total_validation_samples = len(validation_generator1.filenames) + len(validation_generator2.filenames)

# Combine labels from both generators
all_labels = list(train_generator1.classes) + list(train_generator2.classes)

# Get class indices (assuming they are the same for both generators)
class_indices = train_generator1.class_indices
index_to_class = {v: k for k, v in class_indices.items()}

# Count the number of occurrences of each class
class_counts = Counter(all_labels)

# Print the number of images per class
for class_index, count in class_counts.items():
    class_name = index_to_class[class_index]
    print(f"Class '{class_name}' has {count} images.")
    
def combined_generator(gen1, gen2, num_batches):
    while True:
        try:
            batch1_x, batch1_y = next(gen1)
            batch2_x, batch2_y = next(gen2)

            combined_x = np.concatenate((batch1_x, batch2_x), axis=0)
            combined_y = np.concatenate((batch1_y, batch2_y), axis=0)

            yield combined_x, combined_y
        except StopIteration as e:
            print("Generator exhausted or encountered an error:", e)
            break


Found 1120 images belonging to 7 classes.
Found 280 images belonging to 7 classes.
Found 280 images belonging to 7 classes.
Found 1120 images belonging to 7 classes.
Class 'angry' has 320 images.
Class 'disgust' has 320 images.
Class 'fear' has 320 images.
Class 'happy' has 320 images.
Class 'neutral' has 320 images.
Class 'sad' has 320 images.
Class 'surprise' has 320 images.


In [4]:
# Define the number of batches you want from each generator
num_batches_per_gen = 100  # Adjust as needed

# Use the combined generator for training
combined_train_generator = combined_generator(train_generator1, train_generator2, num_batches_per_gen)

# Use the combined generator for validation
combined_validation_generator = combined_generator(validation_generator1, validation_generator2, num_batches_per_gen)


steps_per_epoch = total_train_samples // (2 * 32)
validation_steps = total_validation_samples // (2 * 32)

In [5]:
model = tf.keras.models.load_model(model_name)

In [6]:
# Collect predictions and true labels
y_true = []
y_pred = []

for batch_x, batch_y in combined_validation_generator:
    # Make predictions
    predictions = model.predict(batch_x)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(batch_y, axis=1)
    
    # Append to lists
    y_true.extend(true_classes)
    y_pred.extend(predicted_classes)
    
    # Stop if you have processed the entire validation set
    if len(y_true) >= total_validation_samples:
        break

# Generate classification report
report = classification_report(y_true, y_pred, target_names=validation_generator1.class_indices.keys())
print(report)

2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 70ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step 
              precision    recall  f1-score   support

       angry       0.50      0.57      0.53        80
     disgust       0.65      0.69      0.67        80
        fear       0.40      0.17      0.24        80
       happy       0.96      0.82      0.89        80
     neutral       0.33      0.36      0.34        80
         sad       0.31      0.51      0.39        80
    surprise       0.95      0.70      0.81        80

    accuracy                           0.55       560
   macro avg       0.58      0.55      0.55       560
weighted avg       0.58      0.55      0.55       560



In [7]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report

# Assuming you have the model and validation generators defined
# validation_generator1 and validation_generator2
# Also assuming the target_names are the same for both generators

# Function to generate and print classification report
def generate_classification_report(generator, generator_name):
    y_true = []
    y_pred = []

    for batch_x, batch_y in generator:
        # Generate predictions
        predictions = model.predict(batch_x)
        predicted_classes = np.argmax(predictions, axis=1)
        true_classes = np.argmax(batch_y, axis=1)

        # Collect predictions and true labels
        y_true.extend(true_classes)
        y_pred.extend(predicted_classes)

        # Break if processed all samples
        if len(y_true) >= generator.samples:
            break

    # Generate and print the classification report
    target_names = list(generator.class_indices.keys())
    report = classification_report(y_true, y_pred, target_names=target_names)
    print(f"Classification Report for {generator_name}:\n{report}")

# Generate and print classification reports for both validation generators
generate_classification_report(validation_generator1, "Male Validation Generator")
generate_classification_report(validation_generator2, "Female Validation Generator")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
Classification Report for Male Validation Generator:
              precision    recall  f1-score   support

       angry       0.42      0.53      0.47        40
     disgust       0.42      0.55      0.48        40
        fear       0.61      0.28      0.38        40
       happy       0.94      0.72      0.82        40
     neutral       0.30      0.33      0.31        40
         sad       0.23      0.33      0.27        40
    surprise       0.93      0.70      0.80        40

    accuracy                           0.49       280
   macro avg       0.55      0.49      0.50       280
weighted avg       0.55      0.49      0.50       280

1/1 ━━━━